### Binary semantic segmentation example using U-Net

https://github.com/bnsreenu/python_for_image_processing_APEER/blob/master/tutorial118_binary_semantic_segmentation_using_unet.ipynb

In [1]:
# Libraries

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # ignore tensorflow warnings
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' # allow GPU memory growth

import glob

import cv2
from PIL import Image

import numpy as np
from matplotlib import pyplot as plt

from tensorflow.keras.utils import normalize
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam

In [5]:
import tensorflow as tf
print(tf.test.gpu_device_name())
print('\n')

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/device:GPU:0


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16953317165348376971
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14417788928
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2318304816283472259
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
# Unzip dataset --> run only in Colab
import zipfile

mapai_zip = '/content/drive/MyDrive/prekmurje/prekmurje.zip'
zip_ref = zipfile.ZipFile(mapai_zip, 'r')
zip_ref.extractall("/tmp") # extracting dataset to tmp folder
zip_ref.close()

# Re-run from here after first run! 

In [7]:
# Configure paths

img_dir = '/tmp/prekmurje/train/images'
mask_dir = '/tmp/prekmurje/train/masks'

In [8]:
SIZE = 128
num_images = 46440

#### Load images and masks in order so they match.

#### IMAGES

In [9]:
image_names = glob.glob('/tmp/prekmurje/train/images/*.png')

# Create subset
image_names_subset = image_names[0:num_images]
# Read each subset with cv2
images = [cv2.imread(img, 0) for img in image_names_subset]
# Create dataset
image_dataset = np.array(images)
image_dataset = np.expand_dims(image_dataset, axis = 3)

print(len(image_dataset))

46440


#### MASKS

In [10]:
mask_names = glob.glob('/tmp/prekmurje/train/masks/*.png')
mask_names.sort()
mask_names_subset = mask_names[0:num_images]
masks = [cv2.imread(mask, 0) for mask in mask_names_subset]
mask_dataset = np.array(masks)
mask_dataset = np.expand_dims(mask_dataset, axis = 3)

print(len(mask_dataset))

46440


#### Print properties of images

In [11]:
print("Image data shape is: ", image_dataset.shape)
print("Mask data shape is: ", mask_dataset.shape)
print("Max pixel value in image is: ", image_dataset.max())
print("Labels in the mask are : ", np.unique(mask_dataset))

Image data shape is:  (46440, 128, 128, 1)
Mask data shape is:  (46440, 128, 128, 1)
Max pixel value in image is:  253
Labels in the mask are :  [  0 255]


#### Normalize images
Getting OOM errors, maybe need to find a better way to normalize images.

In [12]:
# IMAGES
# Solution: https://stackoverflow.com/questions/62977311/how-can-i-stop-my-colab-notebook-from-crashing-while-normalising-my-images

image_dataset = (image_dataset / 255.0).astype(np.float16)  #Can also normalize or scale using MinMax scaler

In [14]:
# Need to figure out a way to lower system RAM after normalization
type(mask_dataset)

numpy.ndarray

In [ ]:
# MASKS

#Do not normalize masks, just rescale to 0 to 1.
mask_dataset = (mask_dataset / 255.0).astype(np.float16)   #PIxel values will be 0 or 1

In [ ]:
# 
print(len(image_dataset), len(mask_dataset))

#### Perform train/test split

In [ ]:
# kernel dies

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.20, random_state = 42)

#### Randomly view some images

In [ ]:
import random

image_number = random.randint(0, len(X_train)-1)
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(X_train[image_number,:,:,0], cmap='gray')
plt.subplot(122)
plt.imshow(y_train[image_number,:,:,0], cmap='gray')
plt.show()

### U-Net


In [ ]:
# Building Unet by dividing encoder and decoder into blocks

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate


def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:  #Binary
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model